In [23]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt

model = models.resnet18(pretrained=True)

# implement gradcam from sratch

# for name , m in model.named_parameters():
#     print(name)

# last conv layer is layer4.1.conv2.weight
img_path = r"C:\Users\sharm\Downloads\kuta.png"
img = Image.open(img_path).convert('RGB')


t = T.Compose([
    T.Resize((224 , 224)) ,
    T.ToTensor()
])
img = t(img)
img = img.unsqueeze(0)

In [33]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

pipe = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1).to(device)
pipe.eval()

target_layer = pipe.layer4[-1]

class Gradcam(nn.Module):
    def __init__(self, model, target_layer):
        super().__init__()
        self.model = model
        self.target_layer = target_layer
        
        self.activation = None
        self.grads = None

        self.target_layer.register_forward_hook(self.hook_forward)
        self.target_layer.register_full_backward_hook(self.hook_backward)
        
    def hook_forward(self, module, input, output):
        self.activation = output.detach()
    
    def hook_backward(self, module, grad_in, grad_out):
        self.grads = grad_out[0].detach()
    
    def generate(self, input_tensor, class_idx):
       
        output = self.model(input_tensor)
        self.model.zero_grad()
        
        
        class_score = output[:, class_idx]
        class_score.backward()  
        
        
        weight = torch.mean(self.grads, dim=(2, 3), keepdim=True)
        
        
        cam = torch.sum(weight * self.activation, dim=1, keepdim=True)
        
        
        cam = torch.nn.functional.relu(cam)
        
        
        cam = torch.nn.functional.interpolate(
            cam, size=(224, 224), mode='bilinear', align_corners=False
        )
        
        cam_min = cam.min()
        cam_max = cam.max()
        if cam_max > cam_min:
            cam = (cam - cam_min) / (cam_max - cam_min + 1e-8)
        else:
            cam = torch.zeros_like(cam) 
        
        return cam.squeeze(1).cpu().numpy()  

gcam = Gradcam(pipe, target_layer)

t = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

img_path = r"C:\Users\sharm\Downloads\kuta.png"
img_pil = Image.open(img_path).convert('RGB')
img_tensor = t(img_pil).unsqueeze(0).to(device)  


heatmap_raw = gcam.generate(img_tensor, class_idx=208)
heatmap_2d = heatmap_raw.squeeze(0)  


img_np = np.array(img_pil.resize((224, 224))) / 255.0  

random_mask = (heatmap_2d < 0.2).astype(np.float32)
mask_3 = np.repeat(random_mask[:, :, np.newaxis], 3, axis=2)

m = img_np * mask_3


mask = (heatmap_2d > 0.4).astype(np.float32)
mask_3d = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
masked_image = img_np * mask_3d



m_tensor = torch.from_numpy(masked_image).permute(2, 0, 1).float()

norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
m_tensor = norm(m_tensor).unsqueeze(0).to(device)

with torch.no_grad():
    output_masked = pipe(m_tensor)
    probs = torch.nn.functional.softmax(output_masked, dim=1)
    
    conf_251 = probs[0, 251].item()

print(f"Confidence for Class 251: {conf_251 * 100:.4f}%")


m_tensor = torch.from_numpy(m).permute(2, 0, 1).float()

norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
m_tensor = norm(m_tensor).unsqueeze(0).to(device)



with torch.no_grad():
    output_masked = pipe(m_tensor)
    probs = torch.nn.functional.softmax(output_masked, dim=1)
    
    conf_251 = probs[0, 251].item()

print(f"Confidence for Class 251: {conf_251 * 100:.4f}%")



Using device: cuda
Confidence for Class 251: 50.1342%
Confidence for Class 251: 0.0001%
